In [13]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'HUyBRdqMVPglDd2HUrOBxKn5I'
consumer_secret = 'wthHtv5riIAErx4pllYKsTCNIj9AOG0hq6IE9b5vAhygTr78WN'
access_token = '847102917505138688-dg8GVPVWwfrfM9Xj80wuO69JXcENHW4'
access_secret = 'eNT5doWCDogOY5eb0BZPEJ8bqWUox8O2Nl5pBnRRIXGU2'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [14]:
my_file = open('C:/Study_UIC/576/Assignment2/text_corpus.txt', 'r')
file_lines = my_file.read().lower()
my_file.close()

In [15]:
from time import sleep
import numpy as np
import tensorflow as tf
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [16]:
chars= sorted(list(set(file_lines)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(file_lines) - maxlen, step):
    sentences.append(file_lines[i: i + maxlen])
    next_chars.append(file_lines[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


total chars: 39
nb sequences: 371785
Vectorization...


In [17]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128,nb_epoch=1)

    start_index = random.randint(0, len(file_lines) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = file_lines[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
          

        for i in range(140):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        try:    
            api.update_status(generated[:140])
            sleep(360)
        except tweepy.TweepError as e:
            print(e.reason)
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
 27008/371785 [=>............................] - ETA: 613s - loss: 1.3953

In [29]:
print(generated[:140])
print(next_char)

ho, when they see the hours ripe on earthe hobbars;
he id me's finghly; and thus will sie.

king entrenom url:
theags; o, amfer blood washlo
r
